# Cracking Program and Sefety Protection of the Unity Pro V13.1

This notebook starts from the work from Armis.

See: `https://medium.com/tenable-techblog/examining-crypto-and-bypassing-authentication-in-schneider-electric-plcs-m340-m580-f37cf9f3ff34`

In this work Nicholas Miles reverse engineered "the weak, unknown, non-cryptographically sound custom algorithm" implemented to hide the Program and Sefety Protection password.


In [1]:
# Let's choose a password
password = "password"

In [2]:
# The weak, unknown, non-cryptographically sound custom algorithm used by Unity Pro to hide the Program and Sefety Protection password
# All credits to Nicholas Miles
def crypt_pass(password):
    res_str = ''
    local_c0 = 0

    pwd_data = password.encode('utf-16')[2:]
    
    for i in range(0, len(password)):
        local_6 = pwd_data[(i*2)-1]
        local_5 = pwd_data[i*2]

        local_14 = 0
        for j in range(0, i):
            local_14 = local_14 + pwd_data[j*2]

        local_10 = 0
        for k in range(i+1, len(password)):
            local_10 = local_10 + pwd_data[k*2]

        if(local_5/2 < local_6):
            print("enter")
            local_c0 = calc_pass_char((local_5 + 1) * (local_6 + 1) + local_c0 + local_10)
            res_str += chr(local_c0)

        local_c0 = calc_pass_char(local_c0 * i + local_5 * (i+1) + local_6 *(i+2) + local_14 + local_10)
        print(f"local_c0 = {local_c0}")
        res_str += chr(local_c0)

    return res_str

def calc_pass_char(tmp):
    if(int(tmp) % 2 == 0):
        return (int(tmp) % 0x1a) + 0x41
    else:
        if int(tmp) % 0x10 < 10:
            return (int(tmp) % 0x10) + 48
        else: 
            return (int(tmp) % 0x10) + 55
# Or equivalently,
# def calc_pass_char(byte_):
#    return ((byte_ + 1) % 2) * ((byte_ % 0x1a) + 0x41) + (byte_ % 2) * ((((byte_ % 16) % 11) % 2) * ((byte_ % 16) + 48) + ((((byte_ % 16) % 11) + 1) % 2) * ((byte_ % 16) + 55))


print(f"The password {password} '''encrypted''' (hashed) as {crypt_pass(password)}")


local_c0 = 51
local_c0 = 55
local_c0 = 55
local_c0 = 49
local_c0 = 51
local_c0 = 68
local_c0 = 55
local_c0 = 83
The password password '''encrypted''' (hashed) as 37713D7S


Given any plaintext password it is possible to find another password' with same encryption.
In other words, we don't have a Preimage resistance for this custom hash function.

Please find below the code exemple:

In [3]:
import itertools

alphabet = list(range(0x60, 0x7a))

In [4]:
encrypted_pwd = "37713D7S" 

print(f"Searching a collision for the given encrypted password : {encrypted_pwd} (== encrypted_pwd)")

# Assume letters are between 0 and z
l_bound = len(encrypted_pwd)*(0x30) 
h_bound = len(encrypted_pwd)*(0x7a)

enc_i_list = [ord(encrypted_pwd[i]) for i in range(len(encrypted_pwd))]

# This function finds one plaintext password character
def iterate(enc_i_list, P_guess, acc, depth, max_depth):
    if (P_guess < sum(acc) + 0x30): # Assume 1st letter must be between 0 and z
        return None
    if depth == max_depth:
        return acc
    for p in range(0x60, 0x7a):
        if enc_i_list[depth] == calc_pass_char(enc_i_list[depth-1]*depth + P_guess + p*depth):
            acc += [p]
            call = iterate(enc_i_list, P_guess, acc, depth + 1, max_depth)
            if call == None:
                acc = acc[:-1]
            else:
                return call
    return None

def decrypt_pass(encrypted_pwd):
    for P_guess in range(l_bound, h_bound):
        if calc_pass_char(P_guess) == enc_i_list[0]:
            tmp = iterate(enc_i_list, P_guess, [], 1, len(encrypted_pwd))
            if tmp is not None:
                return [P_guess - sum(tmp)] + tmp

result = decrypt_pass(encrypted_pwd)
result = ''.join([chr(i) for i in result])
print(f"Found a collision plaintext password : {result}")

result_encryption = crypt_pass(result)
assert result_encryption == encrypted_pwd

print(f"'''Encryption''' of {result} gives {result_encryption} (== encrypted_pwd)")





Searching a collision for the given encrypted password : 37713D7S (== encrypted_pwd)
Found a collision plaintext password : 8acccob`
local_c0 = 51
local_c0 = 55
local_c0 = 55
local_c0 = 49
local_c0 = 51
local_c0 = 68
local_c0 = 55
local_c0 = 83
'''Encryption''' of 8acccob` gives 37713D7S (== encrypted_pwd)


Can we do better ? Yes, a bit. By assuming that the characters of password are latin we can find all the possible solutions. This heuristic approach is linear in the lentgh of the password (better than the exponential bruteforce approach).

In [5]:
map_ = {}

for i in range(13*2**4):
    map_[i] = calc_pass_char(i)

inv_map = {}
for k, v in map_.items():
    inv_map[v] = inv_map.get(v, []) + [k]

In [10]:
l_bound = len(encrypted_pwd)*(0x60) 
h_bound = len(encrypted_pwd)*(0x7a)


solutions = [[] for i in range(h_bound-l_bound)]

def iterate(enc_i_list, P_guess, P_index, i, max_i):
    global solutions 
    tmp = []
    for p in range(0x60, 0x7a):
        if (P_guess + p*i + enc_i_list[i-1]*i) % 208 in set(inv_map[enc_i_list[i]]):
            tmp += [p]
    solutions[P_index] += [tmp]
    
def decrypt_pass(encrypted_pwd):
    pwd_list = []
    global solutions
    for P_index, P_guess in enumerate(range(l_bound, h_bound)):
        if calc_pass_char(P_guess) == enc_i_list[0]:
            for i in range(1, len(encrypted_pwd)):
                iterate(enc_i_list, P_guess, P_index, i, len(encrypted_pwd))
    
    for P_index, sol in enumerate(solutions):
        if sol is not None:
            cart_prod = itertools.product(*sol)
            for pwd in cart_prod:
                tmp = range(l_bound, h_bound)[P_index] - sum(pwd)
                if tmp >= 0x60 and tmp <= 0x7A:
                    pwd_list += [chr(tmp) + ''.join([chr(i) for i in pwd])]

    return pwd_list

solutions = decrypt_pass(encrypted_pwd)

assert password in solutions